In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import googlemaps
import gmaps
import os
import requests
import shutil

In [2]:
with open('../resources/key.txt') as f:
    token = f.read()
    
gmaps.configure(api_key = token)

In [23]:
input_folder = '../data/railways/Amtrak_Routes/'
input_file = input_folder+'Amtrak_Routes.shp'

output_folder = '../data/output_images/Amtrak_non_cat_3/'
output_csv = output_folder+'ANC3.csv'

update_folder = output_folder+'/set_2/'

#### Import routes

In [4]:
routes = gpd.read_file(input_file)
routes.head(10)

,OBJECTID,NAME,Shape_Leng,ShapeSTLen,geometry
0,1,Acela,7.410105e+05,9.779207e+05,(LINESTRING (-77.01421252564722 38.88359551476...
1,2,Adirondack,6.156754e+05,8.437497e+05,(LINESTRING (-73.74197100078742 42.64026759195...
2,3,Auto Train,1.474024e+06,1.776112e+06,(LINESTRING (-81.31769605926888 28.75892312078...
3,4,Blue Water,5.114384e+05,6.939223e+05,(LINESTRING (-87.63609509602139 41.81771709246...
4,5,California Zephyr,4.313889e+06,5.672842e+06,(LINESTRING (-108.5558964471134 39.06262618765...
5,6,Capitol Limited,1.287416e+06,1.703070e+06,(LINESTRING (-77.00406395874353 38.90377238870...
6,7,Cardinal,1.865049e+06,2.409211e+06,(LINESTRING (-80.87038216800727 37.65237912675...
7,8,Carolinian/Piedmont,1.166213e+06,1.473517e+06,(LINESTRING (-80.82749062344509 35.23795489421...
8,9,Cascades,7.556294e+05,1.105895e+06,(LINESTRING (-123.0638470134628 44.04802609166...
9,10,Chicago - St.Louis,4.545767e+05,5.950266e+05,(LINESTRING (-90.20919290538818 38.62454351388...


#### Get points from routes

In [12]:
'''
Get all points
'''

points = []
for i in range(0,len(routes.OBJECTID.tolist())):
    try:
        route = routes.iloc[i]
        g = [m for m in route.geometry]
        for j in range(0,len(g)):
            sec_points = mapping(g[j])["coordinates"]
            for k in range(0,len(sec_points)):
                points.append(sec_points[k])
    except Exception as e:
        print("Skipped route number",i,'because',e)

print(len(points),'points')

Skipped route number 0 because 'LineString' object is not iterable
Skipped route number 1 because 'LineString' object is not iterable
Skipped route number 3 because 'LineString' object is not iterable
Skipped route number 5 because 'LineString' object is not iterable
Skipped route number 6 because 'LineString' object is not iterable
Skipped route number 7 because 'LineString' object is not iterable
Skipped route number 9 because 'LineString' object is not iterable
Skipped route number 11 because 'LineString' object is not iterable
Skipped route number 14 because 'LineString' object is not iterable
Skipped route number 15 because 'LineString' object is not iterable
Skipped route number 17 because 'LineString' object is not iterable
Skipped route number 18 because 'LineString' object is not iterable
Skipped route number 21 because 'LineString' object is not iterable
Skipped route number 23 because 'LineString' object is not iterable
Skipped route number 24 because 'LineString' object is 

In [17]:
'''
Get points for 1 or more routes
'''

rows = [8]

points = []
for i in range(0,len(routes.OBJECTID.tolist())):
    if i in rows:
        try:
            route = routes.iloc[i]
            g = [m for m in route.geometry]
            for j in range(0,len(g)):
                sec_points = mapping(g[j])["coordinates"]
                for k in range(0,len(sec_points)):
                    points.append(sec_points[k])
        except Exception as e:
            print("Skipped route number",i,'because',e)

print(len(points),'points')

10234 points


In [ ]:
'''
Get points for single route/section
'''

row = 0
section = 0

points = []
try:
    route = routes.iloc[row]
    g = [i for i in route.geometry]
    len(g)

    points = mapping(g[section])["coordinates"]
except Exception as e:
    print("Skipped route number",row,'because',e)
    
print(len(points),'points')

In [22]:
'''
Get subselection of points
'''

route_points = []
for point in range(0,len(points),100):
    tmp = points[point]
    route_points.append(tmp)

print(len(route_points),'points')

103 points


#### Create working dataset

In [23]:
'''
Create dataframe
'''

df = pd.DataFrame(route_points, columns=['Longitude','Latitude'])

tmp = list(df.Latitude.tolist())
names = []
for i in range(0,len(tmp)):
    name = str(df.iloc[i].Longitude)+'_'+str(df.iloc[i].Latitude)
    names.append(name)

df['Name'] = names

columns = ['Name','Longitude','Latitude']
df= df[columns]

df.head()

,Name,Longitude,Latitude
0,-123.0638470134628_44.04802609166963,-123.063847,44.048026
1,-123.10804673953896_44.05779527583565,-123.108047,44.057795
2,-123.15816369803602_44.10154325401387,-123.158164,44.101543
3,-123.20147978999476_44.2292928872626,-123.201480,44.229293
4,-123.16478181221127_44.28074264405279,-123.164782,44.280743


In [24]:
'''
Export points
'''

df.to_csv(output_csv,header=True,index=None)

In [24]:
'''
Open csv
'''

df = pd.read_csv(output_csv)
df.head()

,Name,Longitude,Latitude,Catenary
0,-123.0638470134628_44.04802609166963,-123.063847,44.048026,0
1,-123.10804673953896_44.05779527583565,-123.108047,44.057795,0
2,-123.15816369803602_44.10154325401387,-123.158164,44.101543,0
3,-123.20147978999476_44.2292928872626,-123.201480,44.229293,0
4,-123.16478181221127_44.28074264405279,-123.164782,44.280743,0


In [25]:
'''
Plot markers
'''

rows = df.index.tolist()

marker_points = []
for row in rows:
    marker_points.append((df.iloc[row].Latitude,df.iloc[row].Longitude))

# Set map centerpoint
coords = marker_points[int((len(marker_points)/2))]
       
# Define map
m = folium.Map(
    location = coords,
    zoom_start = 7
)
  
# Add points    
for mp in marker_points:
    folium.Marker(mp).add_to(m)
m

In [26]:
'''
Get satellite preview for image
'''

row = 0

figure_layout = {
    'width': '100%',
    'height': '800px'
}
fig = gmaps.figure(center = (df.iloc[row].Latitude,df.iloc[row].Longitude), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)
fig

Figure(layout=FigureLayout(height='800px', width='100%'))

In [26]:
'''
Get all images 
'''

root = os.path.dirname(os.path.abspath('Africa.ipynb'))
image_folder = root[:-9]+update_folder[3:]
image_folder

url = 'https://maps.googleapis.com/maps/api/staticmap?'
rows = df.index.tolist()
for i in range(0,len(rows)):
    row = df.iloc[i]
    center = str(row.Latitude)+','+str(row.Longitude)
    payload = {
        'center': center, 
        'zoom': 20,
        'size': '640x640',
        'scale': 2,
        'format': 'png32',
        'maptype': 'satellite',
        'key': token
    }
    
    r = requests.get(url,params=payload,stream=True)

    name = df.iloc[i]['Name']
    filename = image_folder+name+'.png'
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 